In [2]:
# Import dependencies
import pandas as pd
import numpy as np
import time
from itertools import islice

In [6]:
# Create DataFrame from CSV
t0 = time.time()
filtered_lyric_TF_df = pd.read_csv('../../Data/filtered_lyric_TF.csv')
#filtered_lyric_TF_df = filtered_lyric_TF_df.drop([0,1])
filtered_lyric_TF_df.index = filtered_lyric_TF_df.index - 2
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_TF_df.head()

Run time: 40.63580918312073 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
-2,8015,8015,8015,7592.0,8015,8015.0000,8015.0000,7150.0,5501.0,8015.0000,...,5.0,11.0,385.0,7.0,9.0,6.0,24.0,8.0,34.0,5.0
-1,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,5500.0,664.1198,...,5.0,14.0,681.0,7.0,10.0,7.0,31.0,7.0,49.0,4.0
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.6620,0.4130,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [8]:
# Create a new DataFrame and combine the columns for the singular and plural n-word
new_lyric_TF_df = filtered_lyric_TF_df.drop([-2, -1])
for i in range(len(new_lyric_TF_df)):
    new_lyric_TF_df.at[i, 'nigga'] = new_lyric_TF_df.at[i, 'nigga'] + new_lyric_TF_df.at[i, 'niggas']
new_lyric_TF_df = new_lyric_TF_df.drop('niggas', axis=1)

In [9]:
# Create a list of the overall top ten words 
word_count_df = filtered_lyric_TF_df[1:2][filtered_lyric_TF_df.columns[17:]]
word_count_df = word_count_df.sort_values(by=[-1], axis=1, ascending=False)
top_word_counts = list(word_count_df.loc[-1])[0:10]
top_ten = dict(zip(list(word_count_df)[0:10], top_word_counts))
top_ten

{'im': 25280.0,
 'dont': 17907.0,
 '?': 16604.0,
 'like': 16012.0,
 'know': 15030.0,
 'oh': 14976.0,
 'yeah': 14800.0,
 'love': 14119.0,
 'got': 11280.0,
 'get': 10844.0}

In [10]:
# Create a dictionary of the top ten words for each category
t0 = time.time()
top_words = {}
df_rows = []
blues_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='blues'].copy()
blues_df.loc[0] = blues_df.sum(numeric_only=True)
blues_words = blues_df.sort_index()[blues_df.columns[17:]][0:1]
blues_words = blues_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(blues_words.loc[0])[0:10]
top_ten = dict(zip(list(blues_words)[0:10], top_word_counts))
top_words['blues'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

classical_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='classical'].copy()
classical_df.loc[0] = classical_df.sum(numeric_only=True)
classical_words = classical_df.sort_index()[classical_df.columns[17:]][0:1]
classical_words = classical_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(classical_words.loc[0])[0:10]
top_ten = dict(zip(list(classical_words), top_word_counts))
top_words['classical'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

country_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='country'].copy()
country_df.loc[0] = country_df.sum(numeric_only=True)
country_words = country_df.sort_index()[country_df.columns[17:]][0:1]
country_words = country_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(country_words.loc[0])[0:10]
top_ten = dict(zip(list(country_words)[0:10], top_word_counts))
top_words['country'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

funk_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='funk'].copy()
funk_df.loc[0] = funk_df.sum(numeric_only=True)
funk_words = funk_df.sort_index()[funk_df.columns[17:]][0:1]
funk_words = funk_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(funk_words.loc[0])[0:10]
top_ten = dict(zip(list(funk_words)[0:10], top_word_counts))
top_words['funk'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

hiphop_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='hiphop'].copy()
hiphop_df.loc[0] = hiphop_df.sum(numeric_only=True)
hiphop_words = hiphop_df.sort_index()[hiphop_df.columns[17:]][0:1]
hiphop_words = hiphop_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(hiphop_words.loc[0])[0:10]
top_ten = dict(zip(list(hiphop_words)[0:10], top_word_counts))
top_words['hiphop'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

indie_alt_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='indie_alt'].copy()
indie_alt_df.loc[0] = indie_alt_df.sum(numeric_only=True)
indie_alt_words = indie_alt_df.sort_index()[indie_alt_df.columns[17:]][0:1]
indie_alt_words = indie_alt_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(indie_alt_words.loc[0])[0:10]
top_ten = dict(zip(list(indie_alt_words)[0:10], top_word_counts))
top_words['indie_alt'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

jazz_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='jazz'].copy()
jazz_df.loc[0] = jazz_df.sum(numeric_only=True)
jazz_words = jazz_df.sort_index()[jazz_df.columns[17:]][0:1]
jazz_words = jazz_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(jazz_words.loc[0])[0:10]
top_ten = dict(zip(list(jazz_words)[0:10], top_word_counts))
top_words['jazz'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

metal_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='metal'].copy()
metal_df.loc[0] = metal_df.sum(numeric_only=True)
metal_words = metal_df.sort_index()[metal_df.columns[17:]][0:1]
metal_words = metal_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(metal_words.loc[0])[0:10]
top_ten = dict(zip(list(metal_words)[0:10], top_word_counts))
top_words['metal'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

pop_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='pop'].copy()
pop_df.loc[-1] = pop_df.sum(numeric_only=True)
pop_words = pop_df.sort_index()[pop_df.columns[17:]]
pop_words = pop_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(pop_words.loc[0])[0:10]
top_ten = dict(zip(list(pop_words)[0:10], top_word_counts))
top_words['pop'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

punk_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='punk'].copy()
punk_df.loc[0] = punk_df.sum(numeric_only=True)
punk_words = punk_df.sort_index()[punk_df.columns[17:]][0:1]
punk_words = punk_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(punk_words.loc[0])[0:10]
top_ten = dict(zip(list(punk_words)[0:10], top_word_counts))
top_words['punk'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

rnb_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='rnb'].copy()
rnb_df.loc[0] = rnb_df.sum(numeric_only=True)
rnb_words = rnb_df.sort_index()[rnb_df.columns[17:]][0:1]
rnb_words = rnb_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(rnb_words.loc[0])[0:10]
top_ten = dict(zip(list(rnb_words)[0:10], top_word_counts))
top_words['rnb'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

rock_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='rock'].copy()
rock_df.loc[0] = rock_df.sum(numeric_only=True)
rock_words = rock_df.sort_index()[rock_df.columns[17:]][0:1]
rock_words = rock_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(rock_words.loc[0])[0:10]
top_ten = dict(zip(list(rock_words)[0:10], top_word_counts))
top_words['rock'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

romance_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='romance'].copy()
romance_df.loc[0] = romance_df.sum(numeric_only=True)
romance_words = romance_df.sort_index()[romance_df.columns[17:]][0:1]
romance_words = romance_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(romance_words.loc[0])[0:10]
top_ten = dict(zip(list(romance_words)[0:10], top_word_counts))
top_words['romance'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

soul_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='soul'].copy()
soul_df.loc[0] = soul_df.sum(numeric_only=True)
soul_words = soul_df.sort_index()[soul_df.columns[17:]][0:1]
soul_words = soul_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(soul_words.loc[0])[0:10]
top_ten = dict(zip(list(soul_words)[0:10], top_word_counts))
top_words['soul'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 5.531065940856934 seconds


In [11]:
# Print the top words for each category
top_words

{'blues': {'im': 968.0,
  'baby': 809.0,
  'dont': 768.0,
  'love': 644.0,
  'know': 640.0,
  'got': 640.0,
  'oh': 486.0,
  '?': 455.0,
  'well': 374.0,
  'man': 350.0},
 'classical': {'im': 126.0,
  'one': 109.0,
  'come': 105.0,
  'love': 96.0,
  '!': 89.0,
  'dont': 87.0,
  'get': 86.0,
  '?': 74.0,
  'know': 73.0,
  'like': 71.0},
 'country': {'im': 3013.0,
  'like': 2460.0,
  'dont': 2382.0,
  'love': 2025.0,
  'got': 1959.0,
  'yeah': 1935.0,
  'know': 1908.0,
  'oh': 1887.0,
  'aint': 1712.0,
  '?': 1645.0},
 'funk': {'get': 986.0,
  '!': 820.0,
  'baby': 814.0,
  'love': 783.0,
  'dont': 772.0,
  'oh': 746.0,
  'im': 680.0,
  'yeah': 679.0,
  'got': 638.0,
  '?': 632.0},
 'hiphop': {'im': 5176.0,
  'like': 3895.0,
  'yeah': 3749.0,
  'nigga': 3248.0,
  'got': 2823.0,
  'dont': 2722.0,
  '?': 2676.0,
  'get': 2399.0,
  'know': 2317.0,
  'aint': 2036.0},
 'indie_alt': {'im': 1555.0,
  'oh': 1294.0,
  '?': 1288.0,
  'dont': 1177.0,
  'know': 1026.0,
  'like': 905.0,
  'love': 776

In [13]:
# Create dictionary of top word frequencies
unique_top_words = []
for category in top_words:
    unique_top_words.extend(top_words[category])
unique_top_words = list(set(unique_top_words))

top_word_freq = dict.fromkeys(unique_top_words, 0)
for category in top_words:
    for word in top_words[category]:
        top_word_freq[word]+=1
top_word_freq

{'nigga': 1,
 'let': 1,
 'time': 4,
 'well': 1,
 'hand': 1,
 '?': 13,
 'im': 14,
 'ooh': 1,
 'begging': 1,
 'like': 11,
 'oh': 9,
 'one': 2,
 'go': 2,
 'cant': 1,
 'thats': 1,
 'got': 4,
 'come': 2,
 'yeah': 6,
 'ill': 1,
 'baby': 6,
 'take': 1,
 '!': 4,
 'youre': 4,
 'aint': 2,
 'love': 10,
 'plans': 1,
 'man': 2,
 'see': 1,
 'wreck': 1,
 'never': 2,
 'get': 4,
 'dont': 13,
 'know': 13}

In [14]:
# Print top words that are unique to one genre
dict(filter(lambda key: key[1] == 1, top_word_freq.items()))

{'nigga': 1,
 'let': 1,
 'well': 1,
 'hand': 1,
 'ooh': 1,
 'begging': 1,
 'cant': 1,
 'thats': 1,
 'ill': 1,
 'take': 1,
 'plans': 1,
 'see': 1,
 'wreck': 1}

In [15]:
# Print top words that are unique to two genres
dict(filter(lambda key: key[1] == 2, top_word_freq.items()))

{'one': 2, 'go': 2, 'come': 2, 'aint': 2, 'man': 2, 'never': 2}

In [16]:
# Create DataFrame of top ten words
categories = np.array(['blues', 'blues', 'classical', 'classical', 'country', 'country', 'funk', 
              'funk', 'hiphop', 'hiphop', 'indie_alt', 'indie_alt', 'jazz', 'jazz', 'metal',
              'metal', 'pop', 'pop', 'punk', 'punk', 'rnb', 'rnb', 'rock', 'rock', 
              'romance', 'romance', 'soul', 'soul'])
arrays = [categories, np.array(['words', 'frequencies', 'words', 'frequencies', 'words', 
                       'frequencies', 'words', 'frequencies', 'words', 'frequencies', 
                       'words', 'frequencies', 'words', 'frequencies', 'words',
                       'frequencies', 'words', 'frequencies', 'words', 'frequencies', 
                       'words', 'frequencies', 'words', 'frequencies', 'words', 
                       'frequencies', 'words', 'frequencies'])]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['genre', 'w/f'])
top_ten = pd.DataFrame(df_rows, index=arrays, columns=range(1,11))
top_ten

1     2     3      4      5      6      7        8   \
blues     words           im  baby  dont   love   know    got     oh        ?   
          frequencies    968   809   768    644    640    640    486      455   
classical words           im   one  come   love      !   dont    get        ?   
          frequencies    126   109   105     96     89     87     86       74   
country   words           im  like  dont   love    got   yeah   know       oh   
          frequencies   3013  2460  2382   2025   1959   1935   1908     1887   
funk      words          get     !  baby   love   dont     oh     im     yeah   
          frequencies    986   820   814    783    772    746    680      679   
hiphop    words           im  like  yeah  nigga    got   dont      ?      get   
          frequencies   5176  3895  3749   3248   2823   2722   2676     2399   
indie_alt words           im    oh     ?   dont   know   like   love    youre   
          frequencies   1555  1294  1288   1177   1026    905    776      736   
jazz      words         love    im  dont   know      ?   baby   like      let   
          frequencies    490   351   284    231    223    220    206      184   
metal     words           im     ?     !   cant   dont    see  never      one   
          frequencies   1644  1464  1462    887    874    791    774      760   
pop       words        thats   man  take     im  wreck  plans   hand  begging   
          frequencies     13    13    11     10      8      8      8        8   
punk      words           im     ?  dont   know   like     oh  youre        !   
          frequencies   1619  1181  1025    843    779    764    669      590   
rnb       words         love    oh  yeah     im   baby   dont   know     like   
          frequencies   1871  1817  1779   1612   1407   1394   1374     1166   
rock      words           im     ?  dont     oh   like   know   yeah     time   
          frequencies   3547  2629  2580   2502   1964   1889   1689     1480   
romance   words         love    im  know     oh   baby    ill   dont        ?   
          frequencies   1135   618   596    496    473    459    450      408   
soul      words         love    oh  baby   dont   know     im   yeah        ?   
          frequencies   1689  1210  1194   1105   1058   1023   1016      819   

                         9      10  
blues     words        well    man  
          frequencies   374    350  
classical words        know   like  
          frequencies    73     71  
country   words        aint      ?  
          frequencies  1712   1645  
funk      words         got      ?  
          frequencies   638    632  
hiphop    words        know   aint  
          frequencies  2317   2036  
indie_alt words        time    ooh  
          frequencies   682    623  
jazz      words        come   time  
          frequencies   182    181  
metal     words        like   know  
          frequencies   745    710  
pop       words        know   like  
          frequencies     7      7  
punk      words          go  never  
          frequencies   575    574  
rnb       words           ?  youre  
          frequencies  1104    794  
rock      words        love     go  
          frequencies  1472   1416  
romance   words        time  youre  
          frequencies   398    359  
soul      words        like    get  
          frequencies   650    630

In [14]:
# Save DataFrame to CSV
top_ten.to_csv('../Data/top_words.csv')

# Conclusions about top words
- From the total 140 top ten words from each of the 14 genres, 29 were unique
- 7 out of all the top ten words for each genre are unique to one genre
    - 'cant' is unique to the top words in the metal genre
    - 'let' is unique to the top words in the jazz genre
    - 'nigga' is unique to the hiphop genre
    - 'man' and 'well' are unique to the blues genre
    - 'see' is unique to the top words in the metal genre
    - 'ill' is unique to the top words in the romance genre
- 6 out of all the top ten words for each genre are unique to two genre
    - 'go' is unique to the top words in the punk and rock genres
    - 'aint' is unique to the top words in the country and hiphop genres
    - 'come' is unique to the top words in the classical and jazz genres
    - 'ooh' is unique to the top words in the indie-alt and pop genres
    - 'one' isunique to the top words in the classical and metal genres
    - 'never' is unique to the top words in the metal and punk genres
- '?', 'dont', and 'im' are in the top ten words for every genre
    - 'know' is in the top ten words for every genre except funk
    - 'like' and 'love are in the top ten words for 11 out of 14 genres